In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
from glob import glob

def extract_slices_from_folder(input_folder, output_base_dir):
    # Get all .nii.gz files in the input folder
    nii_files = glob(os.path.join(input_folder, '*.nii.gz'))

    for nii_file in nii_files:
        # Load the NIfTI file
        img = nib.load(nii_file)
        data = img.get_fdata()

        # Get the filename without extension
        base_name = os.path.splitext(os.path.splitext(os.path.basename(nii_file))[0])[0]

        # Create output folders
        folders = [f"{base_name}_x", f"{base_name}_y", f"{base_name}_z"]
        for folder in folders:
            os.makedirs(os.path.join(output_base_dir, folder), exist_ok=True)

        # Extract and save slices
        for axis in range(3):
            for i in range(data.shape[axis]):
                if axis == 0:
                    slice_data = data[i, :, :]
                    out_file = os.path.join(output_base_dir, f"{base_name}_x", f"{base_name}_x_{i:03d}.jpg")
                elif axis == 1:
                    slice_data = data[:, i, :]
                    out_file = os.path.join(output_base_dir, f"{base_name}_y", f"{base_name}_y_{i:03d}.jpg")
                else:
                    slice_data = data[:, :, i]
                    out_file = os.path.join(output_base_dir, f"{base_name}_z", f"{base_name}_z_{i:03d}.jpg")

                plt.imsave(out_file, slice_data, cmap='gray')

        print(f"Slices extracted from {base_name} and saved in folders: {', '.join(folders)}")

    print("All .nii.gz files processed.")

# Usage example
input_folder = r"C:\Users\acer\Desktop\Data_0\Nii_cropped_224"
output_base_dir = r"C:\Users\acer\Desktop\Data_0\Nii_toJPG"
extract_slices_from_folder(input_folder, output_base_dir)